```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Implicit Equal Weights Particle Filter

This notebook implements prototyping and example/demo of the Implicit Equal Weights Particle Filter (IEWPF).


## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pyopencl
import os
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')
matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

#Import our simulator
from SWESimulators import CDKLM16, PlotHelper, Common

from SWESimulators import BathymetryAndICs as BC
from SWESimulators import OceanStateNoise
from SWESimulators import OceanNoiseEnsemble
from SWESimulators import BaseOceanStateEnsemble


In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
cl_queue = pyopencl.CommandQueue(cl_ctx)
print "Using ", cl_ctx.devices[0].name

# Ensemble

We need an ensemble where each particle
- runs an independent ocean model
- drift a drifter
- applies a localized small-scale error
- observes the drifter position

Needs to be done:
- Initialize models (create netcdf with init, add error with amp 10*q0(?), put drifter into a small area of the 
- make useful plots to evaluate the results
    - Suggestion: 3-line [eta, hu, hv] plot, with truth, ensemble (mean field with individual drifters), mean-square diff?
    - 3x3/4x4/5x5 plot of eta from different ensemble members?
    - Standard animation of a single ensemble member.


## Create initial condition for ensemble:

In [ ]:
# DEFINE PARAMETERS

#Coriolis well balanced reconstruction scheme
nx = 40
ny = 40

dx = 4.0
dy = 4.0

dt = 0.05
g = 9.81
r = 0.0

f = 0.05
beta = 0.0

ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2])
boundaryConditions = Common.BoundaryConditions(2,2,2,2)

# Define which cell index which has lower left corner as position (0,0)
x_zero_ref = 2
y_zero_ref = 2

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
dataShapeHi = (ny + ghosts[0]+ghosts[2]+1, 
             nx + ghosts[1]+ghosts[3]+1)

eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
eta0_extra = np.zeros(dataShape, dtype=np.float32, order='C')
hv0 = np.zeros(dataShape, dtype=np.float32, order='C');
hu0 = np.zeros(dataShape, dtype=np.float32, order='C');
waterDepth = 10.0
Hi = np.ones(dataShapeHi, dtype=np.float32, order='C')*waterDepth

# Add disturbance:
if True:
    rel_grid_size = nx*1.0/dx
    BC.addBump(eta0, nx, ny, dx, dy, 0.3, 0.5, 0.05*rel_grid_size, validDomain)
    eta0 = eta0*0.3
    BC.addBump(eta0, nx, ny, dx, dy, 0.7, 0.3, 0.10*rel_grid_size, validDomain)
    eta0 = eta0*(-1.3)
    BC.addBump(eta0, nx, ny, dx, dy, 0.15, 0.8, 0.03*rel_grid_size, validDomain)
    eta0 = eta0*1.0
    BC.addBump(eta0, nx, ny, dx, dy, 0.6, 0.75, 0.06*rel_grid_size, validDomain)
    BC.addBump(eta0, nx, ny, dx, dy, 0.2, 0.2, 0.01*rel_grid_size, validDomain)
    eta0 = eta0*(-0.03)
    BC.addBump(eta0_extra, nx, ny, dx, dy, 0.5, 0.5, 0.4*rel_grid_size, validDomain)
    eta0 = eta0 + 0.02*eta0_extra
    BC.initializeBalancedVelocityField(eta0, Hi, hu0, hv0, f, beta, g, nx, ny, dx ,dy, ghosts)
    eta0 = eta0*0.5


if 'sim' in globals():
    sim.cleanUp()
if 'ensemble' in globals():
    ensemble.cleanUp()
    
q0 = 0.5*dt*f/(g*waterDepth)
print "q0: ", q0
print "[f, g, H]", [f, g, waterDepth]
print "f/gH: ", f/(g*waterDepth)
print "gH/f: ", g*waterDepth/f

reload(CDKLM16)
reload(BaseOceanStateEnsemble)
reload(OceanNoiseEnsemble)
reload(PlotHelper)
sim = CDKLM16.CDKLM16(cl_ctx, eta0, hu0, hv0, Hi, \
                      nx, ny, dx, dy, dt, g, f, r, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=False, \
                      small_scale_perturbation=True, \
                      small_scale_perturbation_amplitude=q0)

ensemble_size = 5
ensemble = OceanNoiseEnsemble.OceanNoiseEnsemble(ensemble_size, cl_ctx)
ensemble.setGridInfoFromSim(sim)
ensemble.setStochasticVariables(initialization_variance_factor=5,
                                small_scale_perturbation_amplitude=q0)
ensemble.init()


oldSchoolAnimation = True
if not oldSchoolAnimation:
    T = 300
    sub_t = 10*dt
    for i in range(T):
        t = ensemble.step(sub_t)
        if i == T-1:
            ensemble.printMaxOceanStates()
            
            
    kskugasfd

fig = plt.figure()
plotter = PlotHelper.EnsembleAnimator(fig, ensemble, trueStateOnly=True)

T = 21
sub_t = 300*dt
def animate(i):
    if (i>0):
        t = ensemble.step(sub_t)
    else:
        t = 0.0

    plotter.plot(ensemble);

    fig.suptitle("Ensemble = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) 

anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
ensemble.plotEnsemble() 

In [ ]:
max_dt = ensemble.findLargestPossibleTimeStep()
print "Largest possible timestep with this case: ", max_dt

# Implementing IEWPF

The IEWPF algorithm for the observation at time $t^m$ is applyed to the model at time $t^{m-1}$.
The algorithm consists of the following steps:
1. Find target weight using only deterministic evolution of the model from $t^{m-1}$ to $t^m$.
- Draw a sample from $\xi \sim N(0,P)$.
- Move particles towards observation using a applying some scaling of $\xi$

Some challenges:
- As long as we have one and only one drifter, constant $H(x,y)$ and double periodic boundary conditions, the linear problem within step 1 has the same matrix for any observation. This matrix should therefore be pre-calculated.
- During the creation of $\xi$ we need the random numbers, so using functionality of the OceanNoiseClass would be nice.


Questions:
- Should the IEWPF subroutines/functions/methods be under its own class? Or under the OceanNoiseClass

## Step 0
Pure deterministic integration of the model.

In [ ]:
t = ensemble.step(dt, apply_stochastic_term=False)

## Step 1

The full covariance matrix can be written as
$$ Q = Q^{1/2} Q^{1/2,T} = U_D U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T U_D^T $$
so that 
$$ HQH^T = H U_D U_{GB} \tilde{Q}^{1/2} \tilde{Q}^{1/2} U_{GB}^T U_D^T  H^T$$


In [ ]:
debug = False

def showMatrices(x, y, title):
    fig = plt.figure(figsize=(4,2))
    plt.subplot(1,2,1)
    plt.imshow(x, origin="lower", interpolation="None")
    plt.subplot(1,2,2)
    plt.imshow(y, origin="lower", interpolation="None")
    plt.suptitle(title)
    
    
def SOAR_Q(a_x, a_y, b_x, b_y, dx, dy, q0, L):
    dist = np.sqrt( dx*dx*(a_x - b_x)**2  +  dy*dy*(a_y - b_y)**2)
    return q0*(1.0 + dist/L)*np.exp(-dist/L)


def createS(ensemble, const_H):
    """
    Create the 2x2 matrix S = (HQH^T + R)^-1
    
    Constant as long as
     - one drifter only,
     - H(x,y) = const, and
     - double periodic boundary conditions
    """
    
    dt = ensemble.dt
    dx = ensemble.dx
    dy = ensemble.dy
    geoBalanceConst = ensemble.g*const_H/(2.0*ensemble.f)
    
    # These should be read from a OceanStateNoise object?
    q0 = ensemble.small_scale_perturbation_amplitude
    L = 0.75*dx 
    
    # Local storage for x and y correlations:
    x_corr = np.zeros((7,7))
    y_corr = np.zeros((7,7))
    tmp_x = np.zeros((7,7))
    tmp_y = np.zeros((7,7))
    
    # Mid_coordinates:
    mid_i, mid_j = 3, 3
    
    # Fill the buffers with U_{GB}^T U_D^T  H^T
    x_corr[mid_j+1, mid_i] = -geoBalanceConst*dt/dy
    x_corr[mid_j-1, mid_i] =  geoBalanceConst*dt/dy
    y_corr[mid_j, mid_i+1] =  geoBalanceConst*dt/dx
    y_corr[mid_j, mid_i-1] = -geoBalanceConst*dt/dx
    if debug: showMatrices(x_corr, y_corr, "$U_{GB}^T U_D^T  H^T$")
    
    # Apply the SOAR function to fill x and y with 7x5 and 5x7 respectively
    # First for x:
    for j,i in [mid_j+1, mid_i], [mid_j-1, mid_i]:
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                tmp_x[b, a] += x_corr[j,i]*SOAR_Q(a, b, i, j, dx, dy, q0, L)
    # Then for y:
    for j,i in [mid_j, mid_i+1], [mid_j, mid_i-1]:
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                #print SOAR_Q(a, b, i, j, dx, dy, q0, L)
                tmp_y[b, a] += y_corr[j,i]*SOAR_Q(a, b, i, j, dx, dy, q0, L)
    if debug: showMatrices(tmp_x, tmp_y, "$Q_{SOAR} U_{GB}^T U_D^T  H^T$")      
            
    # Apply the SOARfunction again to fill the points needed to find drift in (mid_i, mid_j)
    # For both x and y:
    for j,i in [mid_j+1, mid_i], [mid_j-1, mid_i], [mid_j, mid_i-1], [mid_j, mid_i+1]:
        x_corr[j,i] = 0
        y_corr[j,i] = 0
        for b in range(j-2, j+3):
            for a in range(i-2, i+3):
                SOAR_Q_res = SOAR_Q(a, b, i, j, dx, dy, q0, L)
                x_corr[j,i] += tmp_x[b, a]*SOAR_Q_res
                y_corr[j,i] += tmp_y[b, a]*SOAR_Q_res
        if debug: print x_corr[j,i], y_corr[j,i]
    if debug: showMatrices(x_corr, y_corr, "$Q_{SOAR} Q_{SOAR} U_{GB}^T U_D^T  H^T$")
    
    # geostrophic balance:
    x_hu = -geoBalanceConst*(x_corr[mid_j+1, mid_i  ] - x_corr[mid_j-1, mid_i  ])/dy
    x_hv =  geoBalanceConst*(x_corr[mid_j  , mid_i+1] - x_corr[mid_j  , mid_i-1])/dx
    y_hu = -geoBalanceConst*(y_corr[mid_j+1, mid_i  ] - y_corr[mid_j-1, mid_i  ])/dy
    y_hv =  geoBalanceConst*(y_corr[mid_j  , mid_i+1] - y_corr[mid_j  , mid_i-1])/dx 
    
    # Drift
    HQHT = np.matrix([[x_hu*dt, y_hu*dt],[x_hv*dt, y_hv*dt]])    
    if debug: print HQHT
    S_inv = HQHT + ensemble.observation_cov
    if debug: print S_inv
    S = np.linalg.inv(S_inv)
    if debug: print S
    return S
    
S = createS(ensemble, 10.0)
print "S: ", S


We expected $HQH^T$ to be a full $2 \times 2$ matrix, but we see now that the $x$ and $y$ coordinate of the drifter is uncorrelated.

When one thinks about it, this makes sense. Even though we know the $x$ position of a drifter, we would have no knowledge of the $y$ position of it. 

The same argument can be used about the velocities. Can we really say anything about the $hu$ in a cell based on the value of $hv$ in the same cell? From $hu(x_i, y_j)$ alone, we can not say whether all the momentum is in $x$-direction, or if it is just a part of the total momentum, which also has a $y$ component. Even though the geostrophic balance generate rotating velocity fields, we will have no knowledge on where in the rotation we are. 

In [ ]:
def solvingLinearProblem(ensemble, S):
    d = ensemble.getInnovations()
    print d
    print ensemble.getDistances()
    
e = solvingLinearProblem(ensemble, S)